### Multiprocessing

For large jobs, you'll want to use multiprocessing or multithreading to speed up your process. This is left up to your preferred method, either within Python or by spawning multiple scripts from the command line. Here are two approaches that I like.

#### Dask

Dask offers easy multithreading (shared resources) and multiprocessing (separate processes) in Python, and is particularly convenient because it includes a subset of Pandas DataFrames.

Here is a minimal example, that lazily loads token frequencies from a list of volume IDs, and counts them up by part of speech tag.

```python
import dask.dataframe as dd
from dask import delayed

def get_tokenlist(vol):
    ''' Load a one volume feature reader, get that volume, and return its tokenlist '''
    return FeatureReader(ids=[volid]).first().tokenlist()

delayed_dfs = [delayed(get_tokenlist)(volid) for volid in volids]

# Create a dask
ddf = (dd.from_delayed(delayed_dfs)
         .reset_index()
         .groupby('pos')[['count']]
         .sum()
      )

# Run processing
ddf.compute()
```

Here is an example of 78 volumes being processed in 24 seconds with 31 threads:

![Counting POS in 78 books about knitting](../data/dask-progress.png)

This example used multithreading. Due to the nature of Python, certain functions won't parallelize well. In our case, the part where the JSON is read from the file and converted to a DataFrame (the light green parts of the graphic) won't speed up because Python dicts lock the Global Interpreter Lock (GIL). However, because Pandas releases the GIL, nearly everything you do after parsing the JSON will be very quick.

To better understand what happens when `ddf.compute()`, here is a graph for 4 volumes:

![](../data/dask-graph.png)


#### GNU Parallel
As an alternative to multiprocessing in Python, my preference is to have simpler Python scripts and to use GNU Parallel on the command line. To do this, you can set up your Python script to take variable length arguments of feature file paths, and to print to stdout.

This psuedo-code shows how that you'd use parallel, where the number of parallel processes is 90% the number of cores, and 50 paths are sent to the script at a time (if you send too little at a time, the initialization time of the script can add up).

```bash
find feature-files/ -name '*json.bz2' | parallel --eta --jobs 90% -n 50 python your_script.py >output.txt
```